In [47]:
# .\.venv\Scripts\Activate.ps1
# pip install -r requirements.txt

In [48]:
import openai
import json
import pandas as pd
import os
import re
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Get the API key from the environment variables
api_key = os.getenv("OPENAI_API_KEY")

# Check if the API key was found and raise an error if not
if not api_key:
    raise ValueError("OpenAI API key not found. Please make sure OPENAI_API_KEY is set in your .env file.")


# Define the part we want to machine
# This is a natural language description that serves as the input to our process
part_description = """
Part Name: Simple Mounting Bracket
Material: Aluminum 6061-T6
Dimensions: Rectangular block, 100mm long, 50mm wide, 20mm thick.
Features:
1. The top surface (100x50mm) needs to be faced to achieve a clean, flat finish.
2. Four (4) M6 through-holes need to be drilled. The holes are located at each corner, 10mm in from each edge.
The raw stock is slightly larger than the final dimensions.     
"""

# Print a confirmation message
print("Part description has been defined.")

Part description has been defined.


In [49]:
def get_process_plan_from_llm(description: str) -> str:
    """
    Calls the LLM API to generate a suggested process plan based on the part description.

    Args:
        description: A natural language description of the part.

    Returns:
        The raw string returned by the LLM, expected to be in JSON format, or None if an error occurs.
    """
    # ------------------ Actual API Call Code (Start) ------------------
    # Initialize the OpenAI client using the previously configured API key
    client = openai.OpenAI() 
    
    # Construct a detailed prompt for the LLM
    prompt = f"""
    You are an expert CNC machinist and process planner. Your task is to generate a step-by-step
    process plan for manufacturing a mechanical part based on the provided description.
    
    Part Description:
    ---
    {description}
    ---
    
    Instructions:
    Generate a detailed process plan. The output MUST be a single, valid JSON object that contains one key: "plan".
    The value of "plan" must be an array of objects.
    Each object in the array represents one machining step and must contain the following keys:
    - "step": (Integer) The sequence number of the operation.
    - "operation": (String) The name of the machining operation (e.g., "Setup", "Facing", "Drilling").
    - "tool_description": (String) A description of the tool to be used (e.g., "10mm 4-Flute End Mill", "M6 Tap").
    - "tool_diameter_mm": (Float or null) The diameter of the tool in millimeters.
    - "spindle_speed_rpm": (Integer or null) The recommended spindle speed in RPM for the specified material.
    - "feed_rate_mm_min": (Integer or null) The recommended feed rate in mm/min for the specified material.
    - "notes": (String) Important details, setup instructions, or comments for the machinist.
    
    Do not add any explanation, markdown, or text outside of the JSON object.
    """
    
    # Use a try-except block to handle potential API errors
    try:
        print("Sending a real API request to OpenAI, please wait...")
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an expert CNC machinist and process planner designed to output JSON."},
                {"role": "user", "content": prompt}
            ],
            response_format={"type": "json_object"} # Use JSON mode
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred during the API call: {e}")
        return None
    # ------------------ Actual API Call Code (End) ------------------

# Execute the LLM call
print("\n1. Starting to generate the process plan from the LLM...")
llm_response_str = get_process_plan_from_llm(part_description)

if llm_response_str:
    print("LLM response received successfully.")
    # --- ADD THIS LINE FOR DEBUGGING ---
    print("\n--- RAW LLM RESPONSE ---")
    print(llm_response_str)
    print("------------------------\n")
    # ---------------------------------
else:
    print("Failed to get a response from the LLM. Please check the error message above.")
    



1. Starting to generate the process plan from the LLM...
Sending a real API request to OpenAI, please wait...
LLM response received successfully.

--- RAW LLM RESPONSE ---
{
  "plan": [
    {
      "step": 1,
      "operation": "Setup",
      "tool_description": "Vise or Fixture",
      "tool_diameter_mm": null,
      "spindle_speed_rpm": null,
      "feed_rate_mm_min": null,
      "notes": "Secure the raw aluminum stock in a milling machine vise ensuring it is stable and square."
    },
    {
      "step": 2,
      "operation": "Facing",
      "tool_description": "50mm Face Mill",
      "tool_diameter_mm": 50,
      "spindle_speed_rpm": 1000,
      "feed_rate_mm_min": 200,
      "notes": "Face the top surface to achieve a flat finish; remove approximately 1mm to ensure a clean surface."
    },
    {
      "step": 3,
      "operation": "Drilling",
      "tool_description": "6mm Twist Drill",
      "tool_diameter_mm": 6,
      "spindle_speed_rpm": 1200,
      "feed_rate_mm_min": 300,
 

In [50]:
def post_process_response(response_str: str) -> list:
    """
    Parses the JSON string returned by the LLM and converts it into a Python list.

    Args:
        response_str: The JSON string returned by the LLM.

    Returns:
        A Python list containing the process steps, or an empty list if parsing fails.
    """
    if not response_str:
        print("Error: The received response is empty.")
        return []
    try:
        # The LLM might return a top-level object containing the plan, so we need to extract the list.
        data = json.loads(response_str)
        if "plan" in data and isinstance(data["plan"], list):
            return data["plan"]
        else:
            # This also handles cases where the LLM returns a list directly.
            return data
    except json.JSONDecodeError:
        print("Error: Failed to parse the LLM response as JSON. Please check its format.")
        # More complex logic could be added here, like trying to fix incomplete JSON with regular expressions.
        return []

print("\n2. Post-processing the LLM response...")
parsed_plan = post_process_response(llm_response_str)
print("Post-processing complete. The JSON string has been converted to a Python list.")
# --- ADD THIS LINE FOR DEBUGGING ---
print("\n--- PARSED DATA STRUCTURE ---")
print(f"Type of parsed_plan: {type(parsed_plan)}")
print(parsed_plan)
print("---------------------------\n")
# ---------------------------------
# print("\nParsed Plan:\n", parsed_plan)


2. Post-processing the LLM response...
Post-processing complete. The JSON string has been converted to a Python list.

--- PARSED DATA STRUCTURE ---
Type of parsed_plan: <class 'list'>
[{'step': 1, 'operation': 'Setup', 'tool_description': 'Vise or Fixture', 'tool_diameter_mm': None, 'spindle_speed_rpm': None, 'feed_rate_mm_min': None, 'notes': 'Secure the raw aluminum stock in a milling machine vise ensuring it is stable and square.'}, {'step': 2, 'operation': 'Facing', 'tool_description': '50mm Face Mill', 'tool_diameter_mm': 50, 'spindle_speed_rpm': 1000, 'feed_rate_mm_min': 200, 'notes': 'Face the top surface to achieve a flat finish; remove approximately 1mm to ensure a clean surface.'}, {'step': 3, 'operation': 'Drilling', 'tool_description': '6mm Twist Drill', 'tool_diameter_mm': 6, 'spindle_speed_rpm': 1200, 'feed_rate_mm_min': 300, 'notes': 'Drill four M6 through-holes at the corners, 10mm from each edge. Ensure proper chip removal and cooling.'}, {'step': 4, 'operation': 'De

In [ ]:
# Define our machine and material constraints
user_material = "Aluminum 6061-T6"  # hard-coded for testing

constraints = {
    "material": user_material,   # hard-coded for testing
    "max_spindle_speed_rpm": 8100,
    "max_feed_rate_mm_min": 15000,
    "valid_operations": {
        # "Setup": ["Vise", "Fixture", "Soft Jaws"],

        "Facing": ["Face Mill", "End Mill"],

        "Roughing": ["End Mill"],
        "Finishing": ["End Mill"],

        "Center Drilling": ["Center Drill"],
        "Drilling": ["Drill Bit", "Center Drill"],
        "Reaming": ["Reamer"],
        "Tapping": ["Tap"],

        "Chamfering": ["Chamfer Mill", "Spot Drill"],
        "Deburring": ["Deburring Tool", "End Mill"],
    }
}

SFM_RANGES = {
    "aluminum": (300, 900),
    "steel":    (80, 250),   # mild/carbon steels (e.g., 1018/1045)
    "titanium": (60, 120),   # e.g., Ti-6Al-4V
}

def _norm(s):
    return (s or "").strip().lower()

def _infer_work_material(material_text: str) -> str:
    t = _norm(material_text)
    if "alum" in t: return "aluminum"
    if "titan" in t or t.startswith("ti"): return "titanium"
    if "steel" in t: return "steel"
    return "steel"  

def _extract_diameter_mm(tool_desc: str, fallback):
    t = (tool_desc or "").lower()
    # e.g., "Ø10 mm", "10mm", "dia 10 mm"
    m = re.search(r"(?:ø|dia|diam)?\s*(\d+(?:\.\d+)?)\s*mm", t)
    if m: return float(m.group(1))
    # e.g., "0.375 in"
    m = re.search(r"(\d+(?:\.\d+)?)\s*in", t)
    if m: return float(m.group(1)) * 25.4
    # e.g., "3/8 in"
    m = re.search(r"(\d+)/(\d+)\s*in", t)
    if m: return (float(m.group(1)) / float(m.group(2))) * 25.4
    # bare number fallback (assume mm if plausible)
    m = re.search(r"\b(\d+(?:\.\d+)?)\b", t)
    if m and "mm" not in t and "in" not in t:
        val = float(m.group(1))
        if 0.5 <= val <= 100.0: return val
    return fallback

def _rpm_window_from_sfm(sfm_min: float, sfm_max: float, d_mm: float):
    # RPM ≈ (SFM * 318) / D_mm
    if not d_mm or d_mm <= 0:
        return (0, 10**9)  # skip if diameter unknown
    k = 318.0
    return int(sfm_min * k / d_mm), int(sfm_max * k / d_mm)
    
def validate_plan(plan: list, constraints: dict) -> list:
    """
    Validates each step of the process plan against predefined constraints.

    Args:
        plan: The parsed list of process plan steps.
        constraints: A dictionary containing machine and process limitations.

    Returns:
        The updated plan list with validation results and warning messages.
    """
    validated_plan = []
    for step in plan:
        warnings = []
        
        # 1) Validate & auto-adjust spindle speed / feedrate against machine limits
        op = (step.get("operation") or "").strip().lower()

        max_rpm  = constraints.get("max_spindle_speed_rpm")
        max_feed = constraints.get("max_feed_rate_mm_min")

        if op != "setup": # Only clamp on real cutting steps (not Setup)
            # Spindle speed
            speed = step.get("spindle_speed_rpm")
            if isinstance(speed, (int, float)) and max_rpm is not None:
                if speed > max_rpm:
                    warnings.append(
                        f"Spindle speed {speed} RPM exceeds machine max {max_rpm} RPM — "
                        f"auto-set to machine maximum."
                    )
                    step["spindle_speed_rpm"] = max_rpm

            # Feedrate
            feed = step.get("feed_rate_mm_min")
            if isinstance(feed, (int, float)) and max_feed is not None:
                if feed > max_feed:
                    warnings.append(
                        f"Feed rate {feed} mm/min exceeds machine max {max_feed} mm/min — "
                        f"auto-set to machine maximum."
                    )
                    step["feed_rate_mm_min"] = max_feed
        else:   # Optional: if Setup rows contain numbers, just warn (don’t clamp)
            speed = step.get("spindle_speed_rpm")
            feed  = step.get("feed_rate_mm_min")
            if isinstance(speed, (int, float)) and speed not in (0, None):
                warnings.append("Setup step has a spindle speed set — should be null/blank.")
            if isinstance(feed, (int, float)) and feed not in (0, None):
                warnings.append("Setup step has a feed rate set — should be null/blank.")

        # 2) Validate if SFM range is appropriate
        material_text = constraints.get("material", "steel")  # e.g., "Aluminum 6061-T6"
        work_mat = _infer_work_material(material_text)

        speed = step.get("spindle_speed_rpm")
        tool_desc = step.get("tool_description") or ""
        d_mm = _extract_diameter_mm(tool_desc, step.get("tool_diameter_mm"))

        if isinstance(speed, (int, float)) and speed is not None and d_mm:
            sfm_cfg = SFM_RANGES.get(work_mat)
            if sfm_cfg:
                rpm_min, rpm_max = _rpm_window_from_sfm(sfm_cfg[0], sfm_cfg[1], d_mm)

                # Allow ±25% slack to avoid false positives
                slack = 0.25
                low_bound  = int(rpm_min * (1 - slack))
                high_bound = int(rpm_max * (1 + slack))

                if speed < low_bound or speed > high_bound:
                    warnings.append(
                        f"Spindle speed {speed} RPM is atypical for {work_mat} at D≈{d_mm:.2f} mm "
                        f"(typical ~{rpm_min}-{rpm_max} RPM from SFM {sfm_cfg[0]}–{sfm_cfg[1]})."
                    )

                    # Optional: auto-adjust toward SFM window (commented out by default)
                    # target = min(max(speed, rpm_min), rpm_max)
                    # if constraints.get('max_spindle_speed_rpm'):
                    #     target = min(target, constraints['max_spindle_speed_rpm'])
                    # if target != speed:
                    #     step["spindle_speed_rpm"] = target
                    #     warnings.append(f"Auto-adjusted spindle speed to {target} RPM (within SFM window).")
    
        # 3. Validate tool selection
        op = step.get("operation")
        tool = step.get("tool_description")
        if op in constraints["valid_operations"]:
            # Check if the suggested tool contains a valid keyword
            valid_tools = constraints["valid_operations"][op]
            if not any(vt.lower() in tool.lower() for vt in valid_tools):
                warnings.append(f"The suggested tool '{tool}' may be inappropriate for the operation '{op}'. Recommended: {', '.join(valid_tools)}.")

        # 4. We can add more checks here, e.g., for feed rate, material, etc.
        
        # Add warning messages to the step for review
        step["validation_warnings"] = "; ".join(warnings) if warnings else "OK"
        validated_plan.append(step)
        
    return validated_plan

print("\n3. Applying validation logic...")
validated_plan = validate_plan(parsed_plan, MACHINE_CONSTRAINTS)
print("Validation complete.")



3. Applying validation logic...
Validation complete.


In [52]:
print("\n4. Generating final output...")

# Use pandas to create a DataFrame for a formatted table output
df = pd.DataFrame(validated_plan)

# Adjust column order for better readability
desired_order = [
    "step", "operation", "tool_description", "spindle_speed_rpm", 
    "feed_rate_mm_min", "notes", "validation_warnings"
]
# Filter out columns that don't exist in the DataFrame to avoid errors
existing_columns = [col for col in desired_order if col in df.columns]
df = df[existing_columns]

# Print the final process plan table
print("--- LLM-Assisted CNC Process Plan ---")
print(f"Part: {part_description.strip().splitlines()[0]}")
print(f"Material: {part_description.strip().splitlines()[1]}")
print("-" * 30)

# Use to_string() to ensure all content is displayed
print(df.to_string())


4. Generating final output...
--- LLM-Assisted CNC Process Plan ---
Part: Part Name: Simple Mounting Bracket
Material: Material: Aluminum 6061-T6
------------------------------
   step  operation     tool_description  spindle_speed_rpm  feed_rate_mm_min                                                                                                     notes                                                                                                                                                                                                                  validation_warnings
0     1      Setup      Vise or Fixture                NaN               NaN                 Secure the raw aluminum stock in a milling machine vise ensuring it is stable and square.                                                                                                                                                                                                                                   O